In [4]:
!pip install json -q

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [1]:
import nltk
data = nltk.download('twitter_samples')

In [11]:
path = r'C:\Users\Kebab\AppData\Roaming\nltk_data\corpora\twitter_samples'
print(path)

C:\Users\Kebab\AppData\Roaming\nltk_data\corpora\twitter_samples


In [27]:
import json, random
import numpy as np

def load_tweets(path):    
    '''
    This function reads the email from directory
    Args:
        path: name of the json file
    Returns:
        data: list of tweets in the file
    '''
    direct = r'C:\Users\Kebab\AppData\Roaming\nltk_data\corpora\twitter_samples'
    data = []
    with open(direct+'\\' + path, 'r') as f:
        for line in f:
            data.append(json.loads(line)['text'])
            
    return data
def parse_tweet(tweets):
    '''
    This function parse the tweet to split into words
    Args:
        tweets: list of strings where each string contains a single tweet
    Returns:
        tweet_word: list of lists where each list contains word in single tweet
        
    '''
    tweet_word = []
    for tweet in tweets:
        tweet_word.append({word for word in tweet.lower().split(' ') if not word.startswith('@')})
    return tweet_word
    
def create_dict(data):
    '''
    This function will count the number of times each word occurs in different tweets
    Args:
        data: list of tweet words
    Returns:
        d: dictionary of word, count pairs where count is how tweets contains the word
    '''
    d = {}
    for tweet in data:
        for word in tweet:
            wrd = d.get(word, None)
            if wrd is not None:
                d[word] += 1
            else:
                d[word] = 1
    return d

def predict(tweets):
    '''
    This function performs the naive Bayes calculation
    Args:
        tweets: list of tweets for making prediction
    Returns:
        ans: list of values(0, 1) wether the tweet is + or -.
    '''
    ans = []
    logit_p, logit_n = np.log(0.5), np.log(0.5) 
    for tweet in tweets:
        for word in tweet:
            p_cnt, n_cnt = pos_dic.get(word, 0), neg_dic.get(word, 0)
            if p_cnt == 0 and n_cnt == 0:
                continue
            else:
                logit_p += np.log((p_cnt + 1)/(v + len(pos_dic)))
                logit_n += np.log((n_cnt + 1)/(v + len(neg_dic)))          
        ans.append(1 if logit_p > logit_n else 0)
    return ans


def metrics(neg, pos):
    '''
    This function calculates the scores of the metrics
    Args:
        neg: Prediction of the model for the negative test tweets
        pos: Prediction of the model for the positive test tweets
    Returns:
        dict: precision, recall, f1 scores
    '''
    tp = sum(pos)
    fn = len(pos) - sum(pos)
    fp = sum(neg)
    tn = len(neg) - sum(neg)
    accuracy = (tp + tn)/(tp + tn + fp + fn)
    recall = tp/(tp + fn)
    precision = tp/(tp + fp)
    f1 = 2 * precision * recall / (precision + recall)
    return {"precision":precision, 'recall':recall, 'f1':f1}
     

In [21]:
#Load the tweets from the directory
neg_path = 'negative_tweets.json'
pos_path = 'positive_tweets.json'
neg_tweet = random.sample(load_tweets(neg_path), 500)
pos_tweet = random.sample(load_tweets(pos_path), 500)

#parse each tweet into a list of words
neg_count, pos_count = parse_tweet(neg_tweet), parse_tweet(pos_tweet)
#split the training and testing tweet
neg_train = neg_count[:400]
pos_train = pos_count[:400]
neg_test = neg_count[400:]
pos_test = pos_count[400:]

In [22]:
# do the counting of the words for positive and negative class
pos_dic, neg_dic = create_dict(pos_train,), create_dict(neg_train)


In [23]:
#calculate the total unique words for laplace smoothing
v = len(set(pos_dic.keys()) |  set(neg_dic.keys()))

In [24]:
#make a prediction on the positive and negative test tweets
neg_pred, pos_pred = predict(neg_test), predict(pos_test)


In [28]:
#calculate the metrics
metrics(neg_pred, pos_pred)

{'precision': 1.0, 'recall': 1.0, 'f1': 1.0}